In [ ]:
from os import path, mkdir
from gensim.models import KeyedVectors
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
base = "/content/drive/MyDrive/NLP100/ch08"

fp_model = "GoogleNews-vectors-negative300.bin"
fp_data = "NewsAggregatorDataset/newsCorpora.csv"
model = KeyedVectors.load_word2vec_format(path.join(base, fp_model), binary=True)
names = ["ID", "TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"]
df = pd.read_csv(path.join(base, fp_data), sep="\t", index_col=0, names=names)
df = df[(df["PUBLISHER"] == "Reuters") | (df["PUBLISHER"] == "Huffington Post") | (df["PUBLISHER"] == "Businessweek") | (df["PUBLISHER"] == "Contactmusic.com") | (df["PUBLISHER"] == "Daily Mail")]
df.head()

,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
ID,,,,,,,
13,Europe reaches crunch point on banking union,http://in.reuters.com/article/2014/03/10/eu-ba...,Reuters,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,in.reuters.com,1394470501755
14,ECB FOCUS-Stronger euro drowns out ECB's messa...,http://in.reuters.com/article/2014/03/10/ecb-p...,Reuters,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,in.reuters.com,1394470501948
20,"Euro Anxieties Wane as Bunds Top Treasuries, S...",http://www.businessweek.com/news/2014-03-10/ge...,Businessweek,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,www.businessweek.com,1394470503148
21,Noyer Says Strong Euro Creates Unwarranted Eco...,http://www.businessweek.com/news/2014-03-10/no...,Businessweek,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,www.businessweek.com,1394470503366
30,REFILE-Bad loan triggers key feature in ECB ba...,http://in.reuters.com/article/2014/03/10/euroz...,Reuters,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,in.reuters.com,1394470505070


In [ ]:
def emb(w):
  try:
    res = model[w]
  except:
    res = np.zeros(300)
  return res


for i in tqdm(df.index):
  df.loc[i, range(300)] = np.mean(np.array([emb(word) for word in df.loc[i, "TITLE"].split()]), axis=0)
df.head()

,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
13,Europe reaches crunch point on banking union,http://in.reuters.com/article/2014/03/10/eu-ba...,Reuters,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,in.reuters.com,1394470501755,0.049351,0.000201,-0.038330,0.121242,-0.056732,-0.012022,0.034286,-0.193324,0.127790,0.002834,-0.161848,-0.020665,0.056100,-0.000759,-0.066999,0.055734,0.072222,0.064819,0.079346,-0.026511,0.015974,0.001709,0.018590,-0.009059,0.111067,-0.048898,-0.175363,-0.033626,0.070033,0.052944,0.000148,0.015698,0.040763,...,-0.007337,-0.065656,-0.032366,0.074044,0.001639,0.109061,-0.054757,-0.069946,-0.038504,-0.055594,-0.002407,0.089164,0.053641,-0.094901,-0.022042,-0.003204,-0.002180,-0.028687,-0.112584,0.019767,-0.172991,0.040179,-0.012068,0.014369,-0.049159,0.037877,-0.095498,0.051165,0.026158,0.071795,-0.010908,-0.046997,-0.109288,0.004046,-0.083182,0.126569,-0.054478,-0.012451,0.028931,-0.028669
14,ECB FOCUS-Stronger euro drowns out ECB's messa...,http://in.reuters.com/article/2014/03/10/ecb-p...,Reuters,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,in.reuters.com,1394470501948,0.039058,-0.003512,-0.012064,0.092548,-0.077489,0.020552,0.030285,-0.031964,0.120756,0.036734,-0.090482,-0.094238,0.077148,0.027123,-0.018677,0.051414,0.005305,0.089787,0.039128,-0.029447,0.080548,0.152945,0.066716,-0.009775,-0.011507,-0.041416,-0.073048,0.121019,0.051284,0.005596,-0.011862,-0.104966,0.017146,...,-0.001399,0.054234,-0.080473,0.054751,0.007324,0.098745,-0.141583,-0.152762,-0.018259,-0.102905,-0.004319,0.023362,0.026076,-0.014432,0.028809,0.001352,0.028161,-0.028226,-0.020038,-0.057569,-0.002223,0.009780,0.063660,0.010911,0.024191,0.090257,-0.050894,0.058343,0.006104,0.011775,-0.052731,0.007310,-0.023569,0.019982,-0.080651,0.046855,0.011676,0.011559,-0.016705,0.004096
20,"Euro Anxieties Wane as Bunds Top Treasuries, S...",http://www.businessweek.com/news/2014-03-10/ge...,Businessweek,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,www.businessweek.com,1394470503148,0.066162,0.025629,-0.033984,0.021924,-0.021143,-0.033624,-0.080078,-0.062573,0.066650,0.105396,-0.106066,0.092171,-0.023981,0.053076,-0.003979,-0.097644,0.078131,0.029785,0.066101,0.011011,0.120494,0.188177,0.268335,0.022778,0.035617,-0.027271,-0.138885,-0.138513,0.011063,0.035577,-0.014526,-0.037024,0.128082,...,0.065918,0.052586,-0.014087,0.034543,0.009084,0.116711,-0.227563,-0.068372,0.013934,0.032257,-0.002545,0.128247,0.008813,-0.002466,0.115088,-0.007788,0.021191,-0.041785,-0.032626,0.028235,-0.148071,0.042407,0.055615,-0.016516,0.092395,-0.041870,-0.104028,0.117920,0.010968,0.167236,0.061057,-0.063110,-0.133215,0.011425,-0.087756,0.064355,-0.024351,0.038611,0.123511,-0.048346
21,Noyer Says Strong Euro Creates Unwarranted Eco...,http://www.businessweek.com/news/2014-03-10/no...,Businessweek,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,www.businessweek.com,1394470503366,0.128238,0.032184,-0.071845,-0.035563,-0.037218,0.106316,-0.031945,0.003567,0.016324,0.101617,-0.185689,0.024071,0.046956,-0.106717,-0.101440,-0.044623,0.053589,0.068509,-0.143270,-0.006912,0.072917,0.205404,0.180874,0.126146,-0.058207,0.005317,-0.051202,-0.082723,-0.010529,0.124946,-0.110215,-0.165717,0.005419,...,0.074419,0.086038,0.091851,0.105178,0.027873,0.200507,-0.145399,-0.172960,-0.026299,-0.048503,-0.042982,0.060421,0.012153,0.006404,0.062500,0.026950,-0.021105,-0.021278,0.023007,0.070638,-0.144396,0.108670,0.017075,-0.023302,0.130649,0.107734,-0.066657,0.081787,0.129530,0.043593,-0.035102,0.007785,-0.071330,0.094184,-0.089654,0.030796,-0.002563,0.094089,0.036282,-0.062066
30,REFILE-Bad loan triggers key feature in ECB ba...,http://in.reuters.com/article/2014/03/10/euroz...,Reuters,b,dPhGU51DcrolU

In [ ]:
df["CATEGORY"] = df["CATEGORY"].replace({"b": 0, "t": 1, "e": 2, "m": 3})

df.head()

,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
13,Europe reaches crunch point on banking union,http://in.reuters.com/article/2014/03/10/eu-ba...,Reuters,0,dPhGU51DcrolUIMxbRm0InaHGA2XM,in.reuters.com,1394470501755,0.049351,0.000201,-0.038330,0.121242,-0.056732,-0.012022,0.034286,-0.193324,0.127790,0.002834,-0.161848,-0.020665,0.056100,-0.000759,-0.066999,0.055734,0.072222,0.064819,0.079346,-0.026511,0.015974,0.001709,0.018590,-0.009059,0.111067,-0.048898,-0.175363,-0.033626,0.070033,0.052944,0.000148,0.015698,0.040763,...,-0.007337,-0.065656,-0.032366,0.074044,0.001639,0.109061,-0.054757,-0.069946,-0.038504,-0.055594,-0.002407,0.089164,0.053641,-0.094901,-0.022042,-0.003204,-0.002180,-0.028687,-0.112584,0.019767,-0.172991,0.040179,-0.012068,0.014369,-0.049159,0.037877,-0.095498,0.051165,0.026158,0.071795,-0.010908,-0.046997,-0.109288,0.004046,-0.083182,0.126569,-0.054478,-0.012451,0.028931,-0.028669
14,ECB FOCUS-Stronger euro drowns out ECB's messa...,http://in.reuters.com/article/2014/03/10/ecb-p...,Reuters,0,dPhGU51DcrolUIMxbRm0InaHGA2XM,in.reuters.com,1394470501948,0.039058,-0.003512,-0.012064,0.092548,-0.077489,0.020552,0.030285,-0.031964,0.120756,0.036734,-0.090482,-0.094238,0.077148,0.027123,-0.018677,0.051414,0.005305,0.089787,0.039128,-0.029447,0.080548,0.152945,0.066716,-0.009775,-0.011507,-0.041416,-0.073048,0.121019,0.051284,0.005596,-0.011862,-0.104966,0.017146,...,-0.001399,0.054234,-0.080473,0.054751,0.007324,0.098745,-0.141583,-0.152762,-0.018259,-0.102905,-0.004319,0.023362,0.026076,-0.014432,0.028809,0.001352,0.028161,-0.028226,-0.020038,-0.057569,-0.002223,0.009780,0.063660,0.010911,0.024191,0.090257,-0.050894,0.058343,0.006104,0.011775,-0.052731,0.007310,-0.023569,0.019982,-0.080651,0.046855,0.011676,0.011559,-0.016705,0.004096
20,"Euro Anxieties Wane as Bunds Top Treasuries, S...",http://www.businessweek.com/news/2014-03-10/ge...,Businessweek,0,dPhGU51DcrolUIMxbRm0InaHGA2XM,www.businessweek.com,1394470503148,0.066162,0.025629,-0.033984,0.021924,-0.021143,-0.033624,-0.080078,-0.062573,0.066650,0.105396,-0.106066,0.092171,-0.023981,0.053076,-0.003979,-0.097644,0.078131,0.029785,0.066101,0.011011,0.120494,0.188177,0.268335,0.022778,0.035617,-0.027271,-0.138885,-0.138513,0.011063,0.035577,-0.014526,-0.037024,0.128082,...,0.065918,0.052586,-0.014087,0.034543,0.009084,0.116711,-0.227563,-0.068372,0.013934,0.032257,-0.002545,0.128247,0.008813,-0.002466,0.115088,-0.007788,0.021191,-0.041785,-0.032626,0.028235,-0.148071,0.042407,0.055615,-0.016516,0.092395,-0.041870,-0.104028,0.117920,0.010968,0.167236,0.061057,-0.063110,-0.133215,0.011425,-0.087756,0.064355,-0.024351,0.038611,0.123511,-0.048346
21,Noyer Says Strong Euro Creates Unwarranted Eco...,http://www.businessweek.com/news/2014-03-10/no...,Businessweek,0,dPhGU51DcrolUIMxbRm0InaHGA2XM,www.businessweek.com,1394470503366,0.128238,0.032184,-0.071845,-0.035563,-0.037218,0.106316,-0.031945,0.003567,0.016324,0.101617,-0.185689,0.024071,0.046956,-0.106717,-0.101440,-0.044623,0.053589,0.068509,-0.143270,-0.006912,0.072917,0.205404,0.180874,0.126146,-0.058207,0.005317,-0.051202,-0.082723,-0.010529,0.124946,-0.110215,-0.165717,0.005419,...,0.074419,0.086038,0.091851,0.105178,0.027873,0.200507,-0.145399,-0.172960,-0.026299,-0.048503,-0.042982,0.060421,0.012153,0.006404,0.062500,0.026950,-0.021105,-0.021278,0.023007,0.070638,-0.144396,0.108670,0.017075,-0.023302,0.130649,0.107734,-0.066657,0.081787,0.129530,0.043593,-0.035102,0.007785,-0.071330,0.094184,-0.089654,0.030796,-0.002563,0.094089,0.036282,-0.062066
30,REFILE-Bad loan triggers key feature in ECB ba...,http://in.reuters.com/article/2014/03/10/euroz...,Reuters,0,dPhGU51DcrolU

In [ ]:
if not path.isdir(path.join(base, "70")):
  mkdir(path.join(base, "70"))
df = df.sample(frac=1, random_state=0)
df_train = df[:int(len(df) * 0.8)]
df_train.to_csv(path.join(base, "70/train.csv"))
df_valid = df[int(len(df) * 0.8):int(len(df) * 0.9)]
df_valid.to_csv(path.join(base, "70/valid.csv"))
df_test = df[int(len(df) * 0.9):]
df_test.to_csv(path.join(base, "70/test.csv"))